#  ReRanker 설치
Korean ReRanker 는 아래 허킹페이스에서 모델을 다운로드 받아 사용합니다.
- [ko-Reranker](https://huggingface.co/Dongjin-kr/ko-reranker)


## 1. Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os


In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored


## 2. Depoly ReRanker model (if needed)

In [4]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [5]:
depoly = True

In [6]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

endpoint_name = f"Ko-Reranker-" + time_stamp

In [7]:
%%time

if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'Dongjin-kr/ko-reranker',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        endpoint_name = endpoint_name,                
        initial_instance_count=1, # number of instances
        instance_type='ml.g4dn.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

---------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Accept: ('application/json',)
ContentType: application/json
Endpoint: Ko-Reranker-2024-02-12-06-58-12
CPU times: user 654 ms, sys: 78 ms, total: 732 ms
Wall time: 5min 3s


In [8]:
reranker_endpoint_name = predictor.endpoint
%store reranker_endpoint_name

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Stored 'reranker_endpoint_name' (str)


## 4. Invocation (prediction)

In [9]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f14ea28a050>


In [10]:
deserializer = "application/json"

In [11]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "반품 신청했는데 무료인가요", "text_pair": "반품신청했었는데 그냥 쓸게요, 주문/반품/교환 취소 철회"},
            {"text": "반품 신청했는데 무료인가요", "text_pair": "취소를 했는데 돈이 안들어와요, 환불/취소 문의"}
        ]
    }
)

In [12]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=reranker_endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.05826818197965622}, {'label': 'LABEL_0', 'score': 0.006919026840478182}]
CPU times: user 22 ms, sys: 37 µs, total: 22 ms
Wall time: 902 ms
